# Imposition Semantics: Kit Fine's Truthmaker Counterfactuals

This notebook explores Kit Fine's imposition semantic framework for counterfactual reasoning. The imposition operation creates alternative worlds by imposing states upon existing worlds, providing a truthmaker-based approach to counterfactuals that differs from traditional similarity-based semantics.

We'll examine both **countermodels** (showing failures of classical counterfactual principles) and **theorems** (showing what principles hold) to understand this innovative approach to counterfactual logic.

## Setup and Theory Configuration

In [1]:
# Import required modules for imposition semantics
import sys
from model_checker.jupyter import create_build_example, build_and_check

# Import imposition semantic components
from model_checker.theory_lib.imposition.semantic import ImpositionSemantics
from model_checker.theory_lib.imposition.operators import imposition_operators
from model_checker.theory_lib.imposition import Proposition, ModelStructure

# Define the imposition semantic theory
imposition_theory = {
    "semantics": ImpositionSemantics,
    "proposition": Proposition,
    "model": ModelStructure,
    "operators": imposition_operators,
}

print(f"Loaded imposition theory with Kit Fine's truthmaker semantics")
print(f"Key operators: □→ (would-counterfactual), ◇→ (might-counterfactual)")

Loaded imposition theory with Kit Fine's truthmaker semantics
Key operators: □→ (would-counterfactual), ◇→ (might-counterfactual)


---

## Example 1: Antecedent Strengthening Failure

### Background
One of the most important failures in counterfactual logic is *antecedent strengthening*. Just because "if A were true, C would be true" doesn't mean that "if A and B were true, C would be true". This failure is crucial for avoiding the paradoxes that plague material implication.

### The Argument
- **Premises**: ¬A (A is false), A □→ C (if A were true, C would be true)
- **Invalid Conclusion**: (A ∧ B) □→ C (if A and B were true, C would be true)
- **Why it fails**: The imposition of A∧B creates a different alternative than imposing just A

In [ ]:
# IM_CM_1: Test counterfactual antecedent strengthening
IM_CM_1_example = [
    ['\\neg A', '(A \\boxright C)'],       # Premises
    ['((A \\wedge B) \\boxright C)'],      # Conclusion (invalid!)
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('IM_CM_1', imposition_theory, IM_CM_1_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'IM_CM_1',
        'Antecedent Strengthening',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected!)")

### Result Interpretation
The countermodel demonstrates how imposition semantics avoids the monotonicity problems of material implication. Different states impose different alternative worlds, so strengthening the antecedent can change which worlds are relevant for evaluation.

---

## Example 2: Transitivity Failure

### Background
*Transitivity* of counterfactuals (if A□→B and B□→C, then A□→C) fails in imposition semantics. This captures the intuition that chains of counterfactual reasoning can break down - the world where B holds due to A might be different from the world where we directly evaluate B.

### The Argument
- **Premises**: A □→ B, B □→ C
- **Invalid Conclusion**: A □→ C
- **Why it fails**: The B-world created by imposing A may differ from directly imposing B

In [ ]:
# IM_CM_10: Test counterfactual transitivity
IM_CM_10_example = [
    ['(A \\boxright B)', '(B \\boxright C)'],  # Premises
    ['(A \\boxright C)'],                        # Conclusion (invalid!)
    {
        'N': 3,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': True,  # We expect a countermodel
    }
]

print("Running model checker...")
model = create_build_example('IM_CM_10', imposition_theory, IM_CM_10_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'IM_CM_10',
        'Counterfactual Transitivity',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected!)")

### Result Interpretation
The failure of transitivity shows that imposition semantics treats counterfactuals as highly context-sensitive. The alternative worlds created by different impositions need not chain together in the expected way, reflecting the non-monotonic nature of counterfactual reasoning.

---

## Example 3: Counterfactual Identity (Valid)

### Background
Despite many classical principles failing, *counterfactual identity* (A □→ A) remains valid. This captures the intuition that if A were true, then A would indeed be true - a minimal requirement for any counterfactual logic.

### The Argument
- **Premises**: None (this is a logical law)
- **Valid Conclusion**: A □→ A
- **Why it works**: Imposing A guarantees A holds in the resulting world

In [ ]:
# IM_TH_1: Test counterfactual identity
IM_TH_1_example = [
    [],                      # No premises - this is a theorem
    ['(A \\boxright A)'],    # Conclusion (valid!)
    {
        'N': 4,
        'contingent': False,
        'disjoint': False,
        'non_empty': False,
        'non_null': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (valid)
    }
]

print("Running model checker...")
model = create_build_example('IM_TH_1', imposition_theory, IM_TH_1_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'IM_TH_1',
        'Counterfactual Identity',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Counterfactual Identity")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nImposing A always creates a world where A holds.")
    print("=" * 70)

### Result Interpretation
The validity of counterfactual identity shows that imposition semantics preserves the most basic counterfactual intuition. This provides a foundation for building more complex counterfactual reasoning.

---

## Example 4: Weakened Transitivity (Valid)

### Background
While full transitivity fails, a *weakened form of transitivity* holds in imposition semantics. If A□→B and (A∧B)□→C, then A□→C. This captures a more careful form of counterfactual chaining that respects the context-sensitivity of imposition.

### The Argument
- **Premises**: A □→ B, (A ∧ B) □→ C
- **Valid Conclusion**: A □→ C
- **Why it works**: The conjunction ensures we're considering compatible impositions

In [ ]:
# IM_TH_3: Test weakened transitivity
IM_TH_3_example = [
    ['(A \\boxright B)', '((A \\wedge B) \\boxright C)'],  # Premises
    ['(A \\boxright C)'],                                    # Conclusion (valid!)
    {
        'N': 4,
        'contingent': False,
        'disjoint': False,
        'non_empty': False,
        'non_null': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (valid)
    }
]

print("Running model checker...")
model = create_build_example('IM_TH_3', imposition_theory, IM_TH_3_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'IM_TH_3',
        'Weakened Transitivity',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Weakened Transitivity")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThis careful form of transitivity respects the imposition structure.")
    print("=" * 70)

### Result Interpretation
Weakened transitivity shows that imposition semantics doesn't abandon transitivity entirely but requires more careful formulation. This principle is crucial for valid counterfactual reasoning while avoiding the paradoxes of full transitivity.

---

## Summary

This notebook has explored Kit Fine's imposition semantics for counterfactuals:

### Invalid Principles (Countermodels)
- **Antecedent Strengthening**: Adding to the antecedent can change the conclusion
- **Full Transitivity**: Counterfactual chains don't necessarily compose

### Valid Principles (Theorems)
- **Counterfactual Identity**: A □→ A always holds
- **Weakened Transitivity**: More careful forms of transitivity remain valid

Imposition semantics provides a truthmaker-based approach to counterfactuals that differs fundamentally from similarity-based approaches. By treating counterfactuals through state imposition rather than world similarity, it captures different intuitions about counterfactual reasoning and avoids certain classical paradoxes.